### 特征点检测
特征检测是计算机视觉和图像处理中的一个概念。
它指的是使用计算机提取图像信息，决定每个图像的点是否属于一个图像特征。

特征检测的结果是把图像上的点分为不同的子集，这些子集往往属于孤立的点、连续的曲线或者连续的区域。|

特征检测包括边缘检测, 角检测, 区域检测 和 脊检测.

特征要满足：
1. 特征是唯一的
2. 特征是可追踪的
3. 特征是能比较的

我们发现：
1. 平坦部分很难找到它在原图的位置
2. 边缘相对好找一点，但是也不能确定
3. 角点可以一下找到在原图的位置

哪些点是角点呢？
+ 灰度梯度对应最大值的元素
+ 两条线的交点
+ 极值点（一阶导数最大，二阶导数为0）

#### Harris角点检测
[Harris角点检测](https://blog.csdn.net/weixin_34910922/article/details/119045533)

人眼对角点的识别通常是在一个局部的小区域或小窗口完成的。
如果在各个方向上移动这个特征的小窗口，窗口内区域的灰度发生了较大的变化，那么就认为在窗口内遇到了角点。
如果这个特定的窗口在图像各个方向上移动时，窗口内图像的灰度没有发生变化，那么窗口内就不存在角点；
如果窗口在某一个方向移动时，窗口内图像的灰度发生了较大的变化，而在另一些方向上没有发生变化，那么，窗口内的图像可能就是一条直线的线段。

Harris 角点检测算法分为以下三步：

1. 当窗口同时向 x 和 y 两个方向移动时，计算窗口内部的像素值变化量E(u,v)；
2. 对于每个窗口，都计算其对应的一个角点响应函数R；
3. 然后对该函数进行阈值处理，如果R > threshold，表示该窗口对应一个角点特征。

![Harris角点检测](Harris角点检测.png)

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
img = cv2.imread('chess.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# blockSize移动的窗口，小一点可以更容易检测出角点，ksize：Sobel算子求一阶导
dst = cv2.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)

# 返回的结果是角点响应，每一个像素点都能计算出响应
print(dst.shape, img.shape)
# 设定阈值，判断角点,设置颜色
img[dst > 0.05 * dst.max()] = (0,0,255)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(889, 907) (889, 907, 3)


> Harris角点具有旋转不变性，但是缩放之后，原来的角点可能就不是角点了

#### Shi-Tomasi角点检测
Shi-Tomasi是Harris角点检测的改进.
Harris角点检测计算的稳定性和K有关,而K是一个经验值,不太好设定最佳的K值.
Shi-Tomasi 发现，角点的稳定性其实和矩阵M的较小特征值有关，于是直接用较小的那个特征值作为分数。这样就不用调整k值了。
![shi-Tomasi角点检测](shi-Tomasi角点检测.png)


In [10]:
img = cv2.imread('chess.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# blockSize移动的窗口，小一点可以更容易检测出角点，ksize：Sobel算子求一阶导
corners = cv2.goodFeaturesToTrack(gray, 0, 0.01, 10)

corners = np.int0(corners)

# 返回的不是角点响应而是坐标
for i in corners:
    x, y = i.ravel()
    cv2.circle(img, (x,y),3,(255,0,0),-1)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### SIFT特征
[链接1](https://blog.csdn.net/gyp_666/article/details/109313957)
[链接2](https://blog.csdn.net/zddblog/article/details/7521424)
[链接3](https://www.cnblogs.com/wangguchangqing/p/4853263.html)

SIFT : Scale-invariant feature transform , 尺度不变特征变换
这种描述具有尺度不变性，可在图像中检测出关键点，是一种

**局部特征描述子**


SIFT算法分解为如下四步：

1. 尺度空间极值检测：搜索所有尺度上的图像位置。通过高斯微分函数来识别潜在的对于尺度和旋转不变的兴趣点。
2. 关键点定位：在每个候选的位置上，通过一个拟合精细的模型来确定位置和尺度。关键点的选择依据于它们的稳定程度。
3. 方向确定：基于图像局部的梯度方向，分配给每个关键点位置一个或多个方向。所有后面的对图像数据的操作都相对于关键点的方向、尺度和位置进行变换，从而提供对于这些变换的不变性。
4. 关键点描述：在每个关键点周围的邻域内，在选定的尺度上测量图像局部的梯度。这些梯度被变换成一种表示，这种表示允许比较大的局部形状的变形和光照变化。

SIFT算法是在不同的尺度空间上查找关键点，而尺度空间的获取需要使用高斯模糊来实现,Lindeberg等人已证明高斯卷积核是实现尺度变换的唯一变换核，并且是唯一的线性核。

使用二维的高斯模板达到了模糊图像的目的，但是会因模板矩阵的关系而造成边缘图像缺失，根据高斯函数的可分离性，可对二维高斯模糊函数进行改进。
高斯函数的可分离性是指使用二维矩阵变换得到的效果也可以通过在水平方向进行一维高斯矩阵变换加上竖直方向的一维高斯矩阵变换得到。从计算的角度来看，这是一项有用的特性。

尺度空间理论
尺度空间理论的基本思想是：
在图像信息处理模型中引入一个被视为尺度的参数，
通过连续变化尺度参数获得多尺度下的尺度空间表示序列，
对这些序列进行尺度空间主轮廓的提取，
并以该主轮廓作为一种特征向量，实现边缘、角点检测和不同分辨率上的特征提取等。

尺度空间中各尺度图像的模糊程度逐渐变大，能够模拟人在距离目标由近到远时目标在视网膜上的形成过程。

![尺度空间的表示](尺度空间的表示.png)





#### 高斯金字塔的构建
尺度空间在实现时使用高斯金字塔表示，高斯金字塔的构建分为两部分：

1. 对图像做不同尺度的高斯模糊
2. 对图像做降采样(隔点采样)

图像的金字塔模型是指，将原始图像不断降阶采样，得到一系列大小不一的图像，由大到小，从下到上构成的塔状模型。
原图像为金子塔的第一层，每次降采样所得到的新图像为金字塔的一层(每层一张图像)，每个金字塔共n层。
金字塔的层数根据图像的原始大小和塔顶图像的大小共同决定，其计算公为了让尺度体现其连续性，高斯金字塔在简单降采样的基础上加上了高斯滤波。
将图像金字塔每层的一张图像使用不同参数做高斯模糊，**使得金字塔的每层含有多张高斯模糊图像**，
将金字塔每层多张图像合称为一组(Octave)，金字塔每层只有一组图像，组数和金字塔层数相等，

另外，降采样时，高斯金字塔上一组图像的初始图像(底层图像)是由前一组图像的倒数第三张图像隔点采样得到的。

#### 高斯差分金字塔Difference of Gaussian

尺度归一化的高斯拉普拉斯函数的极大值和极小值同其它的特征提取函数，例如：梯度，Hessian或Harris角特征比较，能够产生最稳定的图像特征。
而又发现高斯差分函数（Difference of Gaussian ，简称DOG算子）与尺度归一化的高斯拉普拉斯函数非常近似。
因此，Lowe使用更高效的高斯差分算子代替拉普拉斯算子进行极值检测

在实际计算时，使用高斯金字塔每组中相邻上下两层图像相减，得到高斯差分图像，进行极值检测。

### 空间极值点检测
关键点是由DOG空间的局部极值点组成的，关键点的初步探查是通过同一组内各DoG相邻两层图像之间比较完成的。为了寻找DoG函数的极值点，每一个像素点要和它所有的相邻点比较，看其是否比它的图像域和尺度域的相邻点大或者小。如图所示，中间的检测点和它同尺度的8个相邻点和上下相邻尺度对应的9×2个点共26个点比较，以确保在尺度空间和二维图像空间都检测到极值点。

![DOG空间极值检测](DOG空间极值检测.png)

由于要在相邻尺度进行比较，如于每组含4层的高斯差分金子塔，只能在中间两层中进行两个尺度的极值点检测，其它尺度则只能在不同组中进行。
为了在每组中检测S个尺度的极值点，则DOG金字塔每组需S+2层图像，而DOG金字塔由高斯金字塔相邻两层相减得到，
则高斯金字塔每组需S+3层图像，实际计算时S在3到5之间。
当然这样产生的极值点并不全都是稳定的特征点，因为某些极值点响应较弱，而且DOG算子会产生较强的边缘响应。

#### 关键点定位
以上方法检测到的极值点是离散空间的极值点，
以下通过拟合三维二次函数来精确确定关键点的位置和尺度，
同时去除低对比度的关键点和不稳定的边缘响应点(因为DoG算子会产生较强的边缘响应)，
以增强匹配稳定性、提高抗噪声能力。

离散空间的极值点并不是真正的极值点。
利用已知的离散空间点插值得到的连续空间极值点的方法叫做子像素插值（Sub-pixel Interpolation）。

#### 消除边缘响应
一个定义不好的高斯差分算子的极值在横跨边缘的地方有较大的主曲率，而在垂直边缘的方向有较小的主曲率。

DOG算子会产生较强的边缘响应，需要剔除不稳定的边缘响应点。获取特征点处的Hessian矩阵，主曲率通过一个2x2 的Hessian矩阵H求出：

#### 关键点方向分配
为了使描述符具有旋转不变性，需要利用图像的局部特征为给每一个关键点分配一个基准方向。
使用图像梯度的方法求取局部结构的稳定方向。
对于在DOG金字塔中检测出的关键点点，采集其所在高斯金字塔图像3σ邻域窗口内像素的梯度和方向分布特征。

#### 关键点特征描述
通过以上步骤，对于每一个关键点，拥有三个信息：位置、尺度以及方向。接下来就是为每个关键点建立一个描述符，用一组向量将这个关键点描述出来，使其不随各种变化而改变，比如光照变化、视角变化等等。这个描述子不但包括关键点，也包含关键点周围对其有贡献的像素点，并且描述符应该有较高的独特性，以便于提高特征点正确匹配的概率。

SIFT描述子是关键点邻域高斯图像梯度统计结果的一种表示。通过对关键点周围图像区域分块，计算块内梯度直方图，生成具有独特性的向量，这个向量是该区域图像信息的一种抽象，具有唯一性。

Lowe建议描述子使用在关键点尺度空间内4*4的窗口中计算的8个方向的梯度信息，共4*4*8=128维向量表征。

#### SIFT的缺点
SIFT在图像的不变特征提取方面拥有无与伦比的优势，但并不完美，仍然存在：

1. 实时性不高。

2. 有时特征点较少。

3. 对边缘光滑的目标无法准确提取特征点。

等缺点，对模糊的图像和边缘平滑的图像，检测出的特征点过少，对圆更是无能为力。近来不断有人改进，其中最著名的有SURF和CSIFT。

![SIFT特征描述](SIFT特征描述.png)

In [4]:
img = cv2.imread('cat.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 创建sift对象
# sift = cv2.xfeatures2d.SIFT_create()
sift = cv2.SIFT_create()

# 进行检测 keypoint
kp = sift.detect(gray)
# print(kp)
# 绘制关键点
cv2.drawKeypoints(gray, kp, img)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 关键点与描述子的区别

关键点：包含了位置、大小、方向
关键点描述子：记录了关键点周围对其有共享的像素的一组向量值，其不受仿射变换，光照等影响。
描述子的作用就是用于特征匹配

（每一个关键点都有一个128维的向量来描述它）

只有关键点的话不好匹配，不同的图的位置大小不一样

In [7]:
img = cv2.imread('cat.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 创建sift对象
# sift = cv2.xfeatures2d.SIFT_create()
sift = cv2.SIFT_create()

# 进行检测 keypoint
kp = sift.detect(gray)

# 计算描述子
kp, des = sift.compute(img, kp)

# 也可以一步到位，计算关键点和描述子
kp, des = sift.detectAndCompute(img, None)

print(len(kp), des.shape)
# print(kp)
# 绘制关键点
cv2.drawKeypoints(gray, kp, img)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

1637 (1637, 128)


#### SURF(Speed Up Robust Features, 加速稳健特征)
